In [1]:
import sys
sys.path.insert(1, "../")

import sqlite3
from time import *
from lmtanalysis.Chronometer import Chronometer
from lmtanalysis.Animal import *
from lmtanalysis.Detection import *
from lmtanalysis.Measure import *
import matplotlib.pyplot as plt
import numpy as np
from lmtanalysis.FileUtil import getFilesToProcess
from lmtanalysis.Event import *
from lmtanalysis.Measure import *
from lmtanalysis.EventTimeLineCache import EventTimeLineCached
import networkx as nx
from lmtanalysis.Util import convert_to_d_h_m_s, getMinTMaxTAndFileNameInput

import networkx as nx


In [2]:
import sqlite3
from time import *

from lmtanalysis.Chronometer import Chronometer
from lmtanalysis.Animal import *
from lmtanalysis.Detection import *
from lmtanalysis.Measure import *
import matplotlib.pyplot as plt
import numpy as np
from lmtanalysis.Event import *
from lmtanalysis.Measure import *
from lmtanalysis.EventTimeLineCache import EventTimeLineCached
import networkx as nx

def flush( connection ):
    ''' flush event in database '''
    deleteEventTimeLineInBase(connection, "Nest3_" )


def reBuildEvent( connection, file, tmin=None, tmax=None , pool = None ):
    '''
    Nest 3
    ''' 
    print("[NEST 3] : Assume that there is no occlusion, does not work with anonymous animals")
    
    if ( pool == None ):
        pool = AnimalPool( )
        pool.loadAnimals( connection )
        pool.loadDetection( start = tmin, end = tmax , lightLoad=True )
        
    if ( len ( pool.getAnimalList() ) != 4 ):
        print( "[NEST3 Cancelled] 4 animals are required to build nest3.")
        return
    
    contact = {}
        
    for idAnimalA in range( 1 , 5 ):
        for idAnimalB in range( 1 , 5 ):
            if idAnimalA != idAnimalB:    
                contact[idAnimalA, idAnimalB] = EventTimeLineCached( connection, file, "Contact", idAnimalA, idAnimalB, minFrame=tmin, maxFrame=tmax ).getDictionnary()

    stopDictionnary = {}
        
    for idAnimalA in range( 1 , 5 ):
        stopDictionnary[idAnimalA] = EventTimeLineCached( 
            connection, file, "Stop", minFrame=tmin, maxFrame=tmax ).getDictionnary()
    
    nest3TimeLine = {}
    
    for idAnimalA in range( 1 , 5 ):
        # the id will be the one excluded from nest.
        nest3TimeLine[idAnimalA] = EventTimeLine( None, "Nest3_" , idA = idAnimalA , loadEvent=False )
    
    pool.loadAnonymousDetection()
    
    animalList = pool.getAnimalList() 
    
    result = {}
    for idAnimalA in range( 1 , 5 ):
        result[idAnimalA] = {}
    
    for t in range( tmin, tmax+1 ):
                
        isNest = False
        
        nbAnimalAtT = 0
        animalDetectedList = []
        
        anonymousDetectionList = pool.getAnonymousDetection( t )
        
        for animal in animalList:
            if t in animal.detectionDictionnary:
                nbAnimalAtT+=1
                animalDetectedList.append( animal )
        
        #print( str(t) + " : " + str( nbAnimalAtT ) )
                    
    
        #print("TEST")
        graph = nx.Graph()
        # add nodes
        for animal in animalDetectedList:
            graph.add_node( animal )
        for animalA in animalDetectedList:
            for animalB in animalDetectedList:
                if animalA != animalB:
                    # add an edge
                    if t in contact[animalA.baseId,animalB.baseId]:
                        graph.add_edge( animalA, animalB )
        
        # check with anonymous detection. Check contact
        if anonymousDetectionList!= None:
            # manage anonymous
            # print( t , "manage anonymous")
            '''
            # load all masks
            for animal in animalDetectedList:
                animal.loadMask( t )
            '''
            
            for detectionA in anonymousDetectionList: # anonymous with anonymous
                for detectionB in anonymousDetectionList: # anonymous with anonymous
                    if detectionA != detectionB:
                        distance = detectionA.getDistanceTo( detectionB )
                        if distance != None:
                            if distance < DISTANCE_CONTACT_MASS_CENTER:
                                graph.add_edge( detectionA, detectionB )
                                # print("Adding edge with mask (det anonymous to det anonymous)")
                    
            for detection in anonymousDetectionList:
                for animal in animalDetectedList:
                    distance = detection.getDistanceTo(animal.getDetectionAt( t ) )
                    if distance != None:
                        if distance < DISTANCE_CONTACT_MASS_CENTER:
                            #if detection.getMask().isInContactWithMask( animal.getDetectionAt ( t ).getMask() ):
                            graph.add_edge( animal, detection )
                            # print("Adding edge with mask")
        
        # list of CC from the biggest to the smallest
        listCC = sorted(nx.connected_components( graph ), key=len, reverse=True)
        
        if ( len( listCC ) == 2 ): # we have 2 groups
            
            # check if animals in the biggest group are stopped.
            allStoppedInBiggestGroup = True
            for animal in list( listCC[0] ):
                if isinstance( animal , Animal ):
                    if not ( t in stopDictionnary[animal.baseId] ):
                        allStoppedInBiggestGroup = False
                        break
                
            if allStoppedInBiggestGroup:
                if ( len( listCC[1] ) == 1 ): # the 2nd group (and the smallest) has only one mouse
                    animal = list(listCC[1])[0]
                    if isinstance( animal , Animal ):                
                        result[ animal.baseId ][ t ] = True
                 
            
    for idAnimalA in range( 1 , 5 ):
            
        # the id will be the one excluded from nest.
        nest3TimeLine[idAnimalA].reBuildWithDictionnary( result[idAnimalA] )
        # remove very small events
        nest3TimeLine[idAnimalA].removeEventsBelowLength( 2 )
        # merge flashing events
        nest3TimeLine[idAnimalA].mergeCloseEvents( 3 )
        nest3TimeLine[idAnimalA].endRebuildEventTimeLine(connection)
        

    
        

#chronoFullBatch = Chronometer("Full batch" )    
files = getFilesToProcess()
for file in files:
    tmin, tmax, text_file = getMinTMaxTAndFileNameInput()
    connection = sqlite3.connect( file )
    pool = AnimalPool( )
    pool.loadAnimals( connection )
    reBuildEvent(connection,file,tmin,tmax)
    print( "Rebuild event finished." )

Enter time information in frame. You can also set in days, hour, minutes
valid entries: 100, 1d, 1.5d, 23.5h, 1d 2h 3m 4s 5f
Starting t : 0
Entry (in frame) : 0
Ending t : 2d
Entry (in frame) : 5184000
Enter file name to save data (.txt will be added) : nests_3_22
Loading animals.
Fields available in lmtanalysis:  ['ID', 'RFID', 'GENOTYPE', 'NAME']
SQL Query: SELECT ID,RFID,NAME,GENOTYPE FROM ANIMAL ORDER BY GENOTYPE
Animal Id:1 Name:A RFID:000022986409 Genotype:HT User1:None
Animal Id:2 Name:B RFID:000022986492 Genotype:HT User1:None
Animal Id:3 Name:C RFID:000022986390 Genotype:WT User1:None
Animal Id:4 Name:D RFID:000022986562 Genotype:WT User1:None
[NEST 3] : Assume that there is no occlusion, does not work with anonymous animals
Loading animals.
Fields available in lmtanalysis:  ['ID', 'RFID', 'GENOTYPE', 'NAME']
SQL Query: SELECT ID,RFID,NAME,GENOTYPE FROM ANIMAL ORDER BY GENOTYPE
Animal Id:1 Name:A RFID:000022986409 Genotype:HT User1:None
Animal Id:2 Name:B RFID:000022986492 Gen

In [17]:
import os
'''
 For the given path, get the List of all files in the directory tree
'''

def flush4( connection ):
    ''' flush event in database '''
    deleteEventTimeLineInBase(connection, "Nest4_" )
    
def flush3( connection ):
    ''' flush event in database '''
    deleteEventTimeLineInBase(connection, "Nest3_" )


def reBuildEvent4( connection, file, tmin, tmax, pool = None ):
    print("[NEST 4] : Assume that there is no occlusion")
    
    if ( pool == None ):
        pool = AnimalPool( )
        pool.loadAnimals( connection )
        pool.loadDetection( start = tmin, end = tmax , lightLoad=True )

    
    #pool.loadDetection( start = tmin, end = tmax )
    
    if ( len ( pool.getAnimalList() ) != 4 ):
        print( "[NEST4 Cancelled] 4 animals are required to build nest 4.")
        return
    
    contact = {}
    
    
    for animal in range( 1 , 5 ):
        for idAnimalB in range( 1 , 5 ):
            if animal != idAnimalB:    
                contact[animal,idAnimalB] = EventTimeLineCached( connection, file, "Contact", animal, idAnimalB, minFrame=tmin, maxFrame=tmax ).getDictionnary() #fait une matrice de tous les contacts à deux possibles
    
    stopDictionnary = {}
        
    for animal in range( 1 , 5 ):
        stopDictionnary[animal] = EventTimeLineCached( connection, file, "Stop", animal, minFrame=tmin, maxFrame=tmax ).getDictionnary()
    
    nest4TimeLine = EventTimeLine( None, "Nest4_" , loadEvent=False )
    pool.loadAnonymousDetection()
    animalList = pool.getAnimalList() 
    result = {}
    
    for t in range( tmin, tmax ):
        isNest = False
        
        nbAnimalAtT = 0
        animalDetectedList = []
        
        anonymousDetectionList = pool.getAnonymousDetection( t )
        
        for animal in animalList:
            if t in animal.detectionDictionnary:                
                animalDetectedList.append( animal )

        graph = nx.Graph()
        # add nodes
        
        for animal in animalDetectedList:
            graph.add_node( animal )
            nbAnimalAtT+=1
            
        for animalA in animalDetectedList:
            for animalB in animalDetectedList:
                if animalA != animalB:
                    # add an edge
                    if t in contact[animalA.baseId,animalB.baseId]:
                        graph.add_edge( animalA, animalB )
        
        # check with anonymous detection. Check contact
        if anonymousDetectionList!= None:
            
            nbAnimalAtT+=len(anonymousDetectionList)
 
            for detectionA in anonymousDetectionList: # anonymous with anonymous
                for detectionB in anonymousDetectionList: # anonymous with anonymous
                    if detectionA != detectionB:
                        distance = detectionA.getDistanceTo( detectionB )
                        if distance != None:
                            if distance < DISTANCE_CONTACT_MASS_CENTER:
                                graph.add_edge( detectionA, detectionB )
                                #print("Adding edge with mask (det anonymous to det anonymous)")
                    
            for detection in anonymousDetectionList:
                for animal in animalDetectedList:
                    distance = detection.getDistanceTo(animal.getDetectionAt( t ) )
                    if distance != None:
                        if distance < DISTANCE_CONTACT_MASS_CENTER:
                            #if detection.getMask().isInContactWithMask( animal.getDetectionAt ( t ).getMask() ):
                            graph.add_edge( animal, detection )
                            #print("Adding edge with mask")
        
        # list of CC from the biggest to the smallest
        #listCC = sorted(nx.connected_components( graph ), key=len, reverse=True)
        
        if nbAnimalAtT == 0:            
            isNest = True
        
        # list of CC from the biggest to the smallest
        listCC = sorted(nx.connected_components( graph ), key=len, reverse=True)
        
        #largestCC = len ( max(nx.connected_components( graph ), key=len) )
  
        if len ( listCC ) == 0 :
            continue
        
        #print( t , len ( listCC[0] ) , nbAnimalAtT )
        
        if len ( listCC[0] ) == nbAnimalAtT:
        
        #if largestCC == nbAnimalAtT :
            
            # check if animals in the nest are stopped.
            allStoppedInBiggestGroup = True
            for animal in animalDetectedList:
                if isinstance( animal , Animal ):
                    if not ( t in stopDictionnary[animal.baseId] ):
                        allStoppedInBiggestGroup = False
                break

            if allStoppedInBiggestGroup:
                isNest= True                     
                     
        if isNest == True:
            #print( "ADD PUNCTUAL")
            result[t] = True;
            
            
            
    nest4TimeLine.reBuildWithDictionnary( result )
    # remove very small events
    nest4TimeLine.removeEventsBelowLength( 2 )
    # merge flashing events
    nest4TimeLine.mergeCloseEvents( 3 )
    nest4TimeLine.endRebuildEventTimeLine(connection)
    
    
    
def reBuildEvent3( connection, file, tmin=None, tmax=None , pool = None ):
    '''
    Nest 3
    ''' 
    print("[NEST 3] : Assume that there is no occlusion, does not work with anonymous animals")
    
    if ( pool == None ):
        pool = AnimalPool( )
        pool.loadAnimals( connection )
        pool.loadDetection( start = tmin, end = tmax , lightLoad=True )
        
    if ( len ( pool.getAnimalList() ) != 4 ):
        print( "[NEST3 Cancelled] 4 animals are required to build nest3.")
        return
    
    contact = {}
        
    for idAnimalA in range( 1 , 5 ):
        for idAnimalB in range( 1 , 5 ):
            if idAnimalA != idAnimalB:    
                contact[idAnimalA, idAnimalB] = EventTimeLineCached( connection, file, "Contact", idAnimalA, idAnimalB, minFrame=tmin, maxFrame=tmax ).getDictionnary()

    stopDictionnary = {}
        
    for idAnimalA in range( 1 , 5 ):
        stopDictionnary[idAnimalA] = EventTimeLineCached( 
            connection, file, "Stop", minFrame=tmin, maxFrame=tmax ).getDictionnary()
    
    nest3TimeLine = {}
    
    for idAnimalA in range( 1 , 5 ):
        # the id will be the one excluded from nest.
        nest3TimeLine[idAnimalA] = EventTimeLine( None, "Nest3_" , idA = idAnimalA , loadEvent=False )
    
    pool.loadAnonymousDetection()
    
    animalList = pool.getAnimalList() 
    
    result = {}
    for idAnimalA in range( 1 , 5 ):
        result[idAnimalA] = {}
    
    for t in range( tmin, tmax ):
                
        isNest = False
        
        nbAnimalAtT = 0
        animalDetectedList = []
        
        anonymousDetectionList = pool.getAnonymousDetection( t )
        
        for animal in animalList:
            if t in animal.detectionDictionnary:
                nbAnimalAtT+=1
                animalDetectedList.append( animal )

        graph = nx.Graph()
        # add nodes
        for animal in animalDetectedList:
            graph.add_node( animal )
        for animalA in animalDetectedList:
            for animalB in animalDetectedList:
                if animalA != animalB:
                    # add an edge
                    if t in contact[animalA.baseId,animalB.baseId]:
                        graph.add_edge( animalA, animalB )
        
        # check with anonymous detection. Check contact
        if anonymousDetectionList!= None:
            
            for detectionA in anonymousDetectionList: # anonymous with anonymous
                for detectionB in anonymousDetectionList: # anonymous with anonymous
                    if detectionA != detectionB:
                        distance = detectionA.getDistanceTo( detectionB )
                        if distance != None:
                            if distance < DISTANCE_CONTACT_MASS_CENTER:
                                graph.add_edge( detectionA, detectionB )
                                # print("Adding edge with mask (det anonymous to det anonymous)")
                    
            for detection in anonymousDetectionList:
                for animal in animalDetectedList:
                    distance = detection.getDistanceTo(animal.getDetectionAt( t ) )
                    if distance != None:
                        if distance < DISTANCE_CONTACT_MASS_CENTER:
                            #if detection.getMask().isInContactWithMask( animal.getDetectionAt ( t ).getMask() ):
                            graph.add_edge( animal, detection )
                            # print("Adding edge with mask")
        
        # list of CC from the biggest to the smallest
        listCC = sorted(nx.connected_components( graph ), key=len, reverse=True)
        
        if ( len( listCC ) == 2 ): # we have 2 groups
            
            # check if animals in the biggest group are stopped.
            allStoppedInBiggestGroup = True
            for animal in list( listCC[0] ):
                if isinstance( animal , Animal ):
                    if not ( t in stopDictionnary[animal.baseId] ):
                        allStoppedInBiggestGroup = False
                        break
                
            if allStoppedInBiggestGroup:
                if ( len( listCC[1] ) == 1 ): # the 2nd group (and the smallest) has only one mouse
                    animal = list(listCC[1])[0]
                    if isinstance( animal , Animal ):                
                        result[ animal.baseId ][ t ] = True
                 
            
    for idAnimalA in range( 1 , 5 ):
            
        # the id will be the one excluded from nest.
        nest3TimeLine[idAnimalA].reBuildWithDictionnary( result[idAnimalA] )
        # remove very small events
        nest3TimeLine[idAnimalA].removeEventsBelowLength( 2 )
        # merge flashing events
        nest3TimeLine[idAnimalA].mergeCloseEvents( 3 )
        nest3TimeLine[idAnimalA].endRebuildEventTimeLine(connection)
    
    

def getListOfFiles(dirName):
  # create a list of file and sub directories
  # names in the given directory
    listOfFile = os.listdir(dirName)
    allFiles = list()
     # Iterate over all the entries
    for entry in listOfFile:
  # Create full path
        fullPath = os.path.join(dirName, entry)
  # If entry is a directory then get the list of files in this directory
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

def main():
    dirName = 'D:\live mouse tracker data\data';
  # Get the list of all files in directory tree at given path
    listOfFiles = getListOfFiles(dirName)
  # Print the files
    for elem in listOfFiles:
        if elem.endswith('sqlite'):
            print('File: ', elem)
            connection = sqlite3.connect( elem )
            pool = AnimalPool( )
            tmin = 0
            tmax = 2592000
            maxT = pool.getMaxDetectionT()
            if ( tmax > maxT ):
                tmax = maxT
            pool.loadAnimals( connection )
            print('Deleting existing Nest4 events')
            flush4(connection)
            reBuildEvent4(connection,elem,tmin,tmax)
            print('Deleting existing Nest3 events')
            flush3(connection)
            reBuildEvent3(connection,elem,tmin,tmax)
    print ("****************")
'''  
#Get the list of all files in directory tree at given path
    
listOfFiles = list()
for (dirpath, dirnames, filenames) in os.walk(dirName):
   listOfFiles += [os.path.join(dirpath, file) for file in filenames]

# Print the files
for elem in listOfFiles:
if elem.endswith('.sqlite'):
print(elem)
'''

if __name__ == '__main__':
     main()

Loading animals.
Fields available in lmtanalysis:  ['ID', 'RFID', 'GENOTYPE', 'NAME']
SQL Query: SELECT ID,RFID,NAME,GENOTYPE FROM ANIMAL ORDER BY GENOTYPE
Animal Id:1 Name:A RFID:000022986552 Genotype:KO User1:None
Animal Id:2 Name:B RFID:000022986495 Genotype:KO User1:None
Animal Id:3 Name:C RFID:000022986530 Genotype:WT User1:None
Deleting event Nest4_ (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Nest4_"
Number of event deleted: 0 
[NEST 4] : Assume that there is no occlusion
Loading animals.
Fields available in lmtanalysis:  ['ID', 'RFID', 'GENOTYPE', 'NAME']
SQL Query: SELECT ID,RFID,NAME,GENOTYPE FROM ANIMAL ORDER BY GENOTYPE
Animal Id:1 Name:A RFID:000022986552 Genotype:KO User1:None
Animal Id:2 Name:B RFID:000022986495 Genotype:KO User1:None
Animal Id:3 Name:C RFID:000022986530 Genotype:WT User1:None
Animal Id:1 Name:A RFID:000022986552 Genotype:KO User1:None : Loading detection.
SELECT FRAMENUMBER, MASS_X, MASS_Y FROM DETECTION WHERE ANIMALI

Number of event deleted: 0 
[NEST 3] : Assume that there is no occlusion, does not work with anonymous animals
Loading animals.
Fields available in lmtanalysis:  ['ID', 'RFID', 'GENOTYPE', 'NAME']
SQL Query: SELECT ID,RFID,NAME,GENOTYPE FROM ANIMAL ORDER BY GENOTYPE
Animal Id:1 Name:A RFID:000022986552 Genotype:KO User1:None
Animal Id:3 Name:C RFID:000022986494 Genotype:KO User1:None
Animal Id:2 Name:B RFID:000022986394 Genotype:WT User1:None
Animal Id:1 Name:A RFID:000022986552 Genotype:KO User1:None : Loading detection.
SELECT FRAMENUMBER, MASS_X, MASS_Y FROM DETECTION WHERE ANIMALID=1 AND FRAMENUMBER>=0 AND FRAMENUMBER<=0
Animal Id:1 Name:A RFID:000022986552 Genotype:KO User1:None   0  detections loaded in 0.0010001659393310547 seconds.
Animal Id:3 Name:C RFID:000022986494 Genotype:KO User1:None : Loading detection.
SELECT FRAMENUMBER, MASS_X, MASS_Y FROM DETECTION WHERE ANIMALID=3 AND FRAMENUMBER>=0 AND FRAMENUMBER<=0
Animal Id:3 Name:C RFID:000022986494 Genotype:KO User1:None   0 

Number of event deleted: 10196 
[NEST 4] : Assume that there is no occlusion
Loading animals.
Fields available in lmtanalysis:  ['ID', 'RFID', 'GENOTYPE', 'NAME']
SQL Query: SELECT ID,RFID,NAME,GENOTYPE FROM ANIMAL ORDER BY GENOTYPE
Animal Id:2 Name:B RFID:000022986532 Genotype:KO User1:None
Animal Id:4 Name:D RFID:RFID_D Genotype:KO User1:None
Animal Id:1 Name:A RFID:000022986514 Genotype:WT User1:None
Animal Id:3 Name:C RFID:000022986389 Genotype:WT User1:None
Animal Id:2 Name:B RFID:000022986532 Genotype:KO User1:None : Loading detection.
SELECT FRAMENUMBER, MASS_X, MASS_Y FROM DETECTION WHERE ANIMALID=2 AND FRAMENUMBER>=0 AND FRAMENUMBER<=0
Animal Id:2 Name:B RFID:000022986532 Genotype:KO User1:None   0  detections loaded in 0.0009999275207519531 seconds.
Animal Id:4 Name:D RFID:RFID_D Genotype:KO User1:None : Loading detection.
SELECT FRAMENUMBER, MASS_X, MASS_Y FROM DETECTION WHERE ANIMALID=4 AND FRAMENUMBER>=0 AND FRAMENUMBER<=0
Animal Id:4 Name:D RFID:RFID_D Genotype:KO User1:N

139816  frames containing anonymous detections loaded in 0.8631925582885742 seconds.
No event in timeLine
no event
Keep previous entry.
Saving timeLine: Nest4_ ( 0 events )
Deleting event Nest3_ (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Nest3_"
Number of event deleted: 10795 
[NEST 3] : Assume that there is no occlusion, does not work with anonymous animals
Loading animals.
Fields available in lmtanalysis:  ['ID', 'RFID', 'GENOTYPE', 'NAME']
SQL Query: SELECT ID,RFID,NAME,GENOTYPE FROM ANIMAL ORDER BY GENOTYPE
Animal Id:1 Name:A RFID:000022986532 Genotype:KO User1:None
Animal Id:4 Name:D RFID:RFID_D Genotype:KO User1:None
Animal Id:2 Name:B RFID:000022986389 Genotype:WT User1:None
Animal Id:3 Name:C RFID:000022986514 Genotype:WT User1:None
Animal Id:1 Name:A RFID:000022986532 Genotype:KO User1:None : Loading detection.
SELECT FRAMENUMBER, MASS_X, MASS_Y FROM DETECTION WHERE ANIMALID=1 AND FRAMENUMBER>=0 AND FRAMENUMBER<=0
Animal Id:1 Name:A RFID:0

878410  frames containing anonymous detections loaded in 6.150388479232788 seconds.
No event in timeLine
no event
Keep previous entry.
Saving timeLine: Nest3_ ( 0 events )
No event in timeLine
no event
Keep previous entry.
Saving timeLine: Nest3_ ( 0 events )
No event in timeLine
no event
Keep previous entry.
Saving timeLine: Nest3_ ( 0 events )
No event in timeLine
no event
Keep previous entry.
Saving timeLine: Nest3_ ( 0 events )
Loading animals.
Fields available in lmtanalysis:  ['ID', 'RFID', 'GENOTYPE', 'NAME']
SQL Query: SELECT ID,RFID,NAME,GENOTYPE FROM ANIMAL ORDER BY GENOTYPE
Animal Id:1 Name:A RFID:000022986511 Genotype:KO User1:None
Animal Id:2 Name:B RFID:RFID_B Genotype:KO User1:None
Deleting event Nest4_ (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Nest4_"
Number of event deleted: 0 
[NEST 4] : Assume that there is no occlusion
Loading animals.
Fields available in lmtanalysis:  ['ID', 'RFID', 'GENOTYPE', 'NAME']
SQL Query: SELECT ID,RFI

Animal Id:4 Name:D RFID:000022986387 Genotype:WT User1:None   0  detections loaded in 0.03400731086730957 seconds.
Contact  Id( 1 , 2 , None , None ) Loaded from cache ( 0  records. )
Contact  Id( 1 , 3 , None , None ) Loaded from cache ( 0  records. )
Contact  Id( 1 , 4 , None , None ) Loaded from cache ( 0  records. )
Contact  Id( 2 , 1 , None , None ) Loaded from cache ( 0  records. )
Contact  Id( 2 , 3 , None , None ) Loaded from cache ( 0  records. )
Contact  Id( 2 , 4 , None , None ) Loaded from cache ( 0  records. )
Contact  Id( 3 , 1 , None , None ) Loaded from cache ( 0  records. )
Contact  Id( 3 , 2 , None , None ) Loaded from cache ( 0  records. )
Contact  Id( 3 , 4 , None , None ) Loaded from cache ( 0  records. )
Contact  Id( 4 , 1 , None , None ) Loaded from cache ( 0  records. )
Contact  Id( 4 , 2 , None , None ) Loaded from cache ( 0  records. )
Contact  Id( 4 , 3 , None , None ) Loaded from cache ( 0  records. )
Stop  Id( 1 , None , None , None ) Loaded from cache ( 0 

621535  frames containing anonymous detections loaded in 32.36134886741638 seconds.
No event in timeLine
no event
Keep previous entry.
Saving timeLine: Nest4_ ( 0 events )
Deleting event Nest3_ (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Nest3_"
Number of event deleted: 32095 
[NEST 3] : Assume that there is no occlusion, does not work with anonymous animals
Loading animals.
Fields available in lmtanalysis:  ['ID', 'RFID', 'GENOTYPE', 'NAME']
SQL Query: SELECT ID,RFID,NAME,GENOTYPE FROM ANIMAL ORDER BY GENOTYPE
Animal Id:1 Name:A RFID:000022986409 Genotype:HT User1:None
Animal Id:2 Name:B RFID:000022986492 Genotype:HT User1:None
Animal Id:3 Name:C RFID:000022986390 Genotype:WT User1:None
Animal Id:4 Name:D RFID:000022986562 Genotype:WT User1:None
Animal Id:1 Name:A RFID:000022986409 Genotype:HT User1:None : Loading detection.
SELECT FRAMENUMBER, MASS_X, MASS_Y FROM DETECTION WHERE ANIMALID=1 AND FRAMENUMBER>=0 AND FRAMENUMBER<=0
Animal Id:1 Name:A R

511024  frames containing anonymous detections loaded in 2.7886557579040527 seconds.
No event in timeLine
no event
Keep previous entry.
Saving timeLine: Nest3_ ( 0 events )
No event in timeLine
no event
Keep previous entry.
Saving timeLine: Nest3_ ( 0 events )
No event in timeLine
no event
Keep previous entry.
Saving timeLine: Nest3_ ( 0 events )
No event in timeLine
no event
Keep previous entry.
Saving timeLine: Nest3_ ( 0 events )
Loading animals.
Fields available in lmtanalysis:  ['ID', 'RFID', 'GENOTYPE', 'NAME']
SQL Query: SELECT ID,RFID,NAME,GENOTYPE FROM ANIMAL ORDER BY GENOTYPE
Animal Id:1 Name:A RFID:000022986396 Genotype:WT User1:None
Animal Id:2 Name:B RFID:000022986530 Genotype:WT User1:None
Animal Id:3 Name:C RFID:000022986394 Genotype:WT User1:None
Deleting event Nest4_ (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Nest4_"
Number of event deleted: 0 
[NEST 4] : Assume that there is no occlusion
Loading animals.
Fields available in lmtana

1081854  frames containing anonymous detections loaded in 6.239075422286987 seconds.
No event in timeLine
no event
Keep previous entry.
Saving timeLine: Nest3_ ( 0 events )
No event in timeLine
no event
Keep previous entry.
Saving timeLine: Nest3_ ( 0 events )
No event in timeLine
no event
Keep previous entry.
Saving timeLine: Nest3_ ( 0 events )
No event in timeLine
no event
Keep previous entry.
Saving timeLine: Nest3_ ( 0 events )
Loading animals.
Fields available in lmtanalysis:  ['ID', 'RFID', 'GENOTYPE', 'NAME']
SQL Query: SELECT ID,RFID,NAME,GENOTYPE FROM ANIMAL ORDER BY GENOTYPE
Animal Id:2 Name:B RFID:000022986495 Genotype:KO User1:None
Animal Id:1 Name:A RFID:000022986396 Genotype:WT User1:None
Animal Id:3 Name:C RFID:000022986394 Genotype:WT User1:None
Deleting event Nest4_ (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Nest4_"
Number of event deleted: 0 
[NEST 4] : Assume that there is no occlusion
Loading animals.
Fields available in lmtana